### COLLOCATIONS

#### Отчет

Задача: На основании этих данных выделить наиболее типичные для суда действия (словосочетания типа принять решение), применив два различных метода.

Для оценки результата:

1) Пользуясь словарями, интуицией и т.п. составляете золотой стандарт – ранжированный топ 10 коллокаций.

> golden_stand = [(('ПРИНЯТЬ','РЕШЕНИЕ'),1), (('ПРИЗНАТЬ','ВИНОВНАЯ'),2), (('ВЫНЕСТИ','ПРИГОВОР'),3), 
                (('НАЗНАЧИТЬ','НАКАЗАНИЕ'),4), (('УДОВЛЕТВОРИТЬ','ХОДАТАЙСТВО'),5), 
                (('УДОВЛЕТВОРИТЬ','ИСК'),6), (('ОТКЛОНИТЬ','ХОДАТАЙСТВО'),7), (('ОТКЛОНИТЬ','ИСК'),8), 
                (('ОБЖАЛОВАТЬ','ПРИГОВОР'),9), (('ПРЕКРАТИТЬ','ДЕЛО'),10)]

2) Смотрите, какую позицию занимают коллокации из золотого стандарта среди списка, отранжированного по мере коллокационной связи.

3) Считаете меру качества – ранговый коэффициент корреляции Спирмена между двумя списками.

Методы:
* PMI
* LR

Итогом работы является скрипт на питоне + список топ-коллокаций, полученных методом 1, список топ-коллокаций, полученных методом 2. Отчет по заданию: (0) какой датасет - корпус, количество токенов; для каждого метода/параметра метода указать, (а) какие коллокации, полученные методом i, не входят в Ваш золотой стандарт, (б) какие попали ошибочно (предложите объяснение - откуда взялась такая ошибка, что можно сделать, чтобы ее устранить), (в) какие у Вас не попоали в ЗС, но вы думаете, что это полезные коллокации.


In [8]:
import pandas as pd
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [9]:
df = pd.read_csv('court-V-N.csv') # corpus
rows = list(map(list, df.values)) # list of rows in corpus
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)

In [10]:
# top 10 -- LR
top_lr = finder.nbest(bigram_measures.likelihood_ratio, 10)

In [11]:
# top 10 -- PMI
top_pmi = finder.nbest(bigram_measures.pmi, 10)

In [12]:
# lists with scores and with scores removed
list_with_scores_lr = list(finder.score_ngrams(bigram_measures.likelihood_ratio))
list_no_scores_lr = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_lr]
list_with_scores_pmi = list(finder.score_ngrams(bigram_measures.pmi))
list_no_scores_pmi = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_pmi]

In [20]:
golden_stand = [(('ПРИНЯТЬ','РЕШЕНИЕ'),1), (('ПРИЗНАТЬ','ВИНОВНАЯ'),2), (('ВЫНЕСТИ','ПРИГОВОР'),3), 
                (('НАЗНАЧИТЬ','НАКАЗАНИЕ'),4), (('УДОВЛЕТВОРИТЬ','ХОДАТАЙСТВО'),5), 
                (('УДОВЛЕТВОРИТЬ','ИСК'),6), (('ОТКЛОНИТЬ','ХОДАТАЙСТВО'),7), (('ОТКЛОНИТЬ','ИСК'),8), 
                (('ОБЖАЛОВАТЬ','ПРИГОВОР'),9), (('ПРЕКРАТИТЬ','ДЕЛО'),10)]

ranks_golden = [p[1] for p in golden_stand]
ranks_lr = [list_no_scores_lr.index(i[0]) for i in golden_stand]
ranks_pmi = [list_no_scores_pmi.index(i[0]) for i in golden_stand]
print('gold', ranks_golden)
print('lr', ranks_lr)
print('pmi', ranks_pmi)

gold [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lr [5, 12, 86, 1992, 4, 3, 1251, 80, 2123, 1648]
pmi [1976, 1963, 1978, 1792, 1924, 1997, 2123, 2062, 1927, 2085]


In [24]:
from nltk.metrics.spearman import *
print(golden_stand, ranks_lr)
print(spearman_correlation(golden_stand, ranks_lr))

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [15]:
spearmanr(ranks_golden,ranks_lr)

SpearmanrResult(correlation=0.45454545454545447, pvalue=0.18690481034933201)

In [18]:
spearmanr(ranks_golden,ranks_pmi)

SpearmanrResult(correlation=0.43030303030303024, pvalue=0.21449233253280775)